<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Interfaces_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interfaces in Python: Protocols and ABCs

Source: http://masnun.rocks/2017/04/15/interfaces-in-python-protocols-and-abcs/ 


The idea of interface is really simple - it is the description of how an object behaves. An interface tells us what an object can do to play it’s role in a system. In object oriented programming, an interface is a set of publicly accessible methods on an object which can be used by other parts of the program to interact with that object. Interfaces set clear boundaries and help us organize our code better. In some langauges like Java, interfaces are part of the language syntax and strictly enforced. However, in Python, things are a little different. In this post, we will explore how interfaces can be implemented in Python.

## Informal Interfaces: Protocols / Duck Typing
There’s no interface keyword in Python. The Java / C# way of using interfaces is not available here. In the dynamic language world, things are more implicit. We’re more focused on how an object behaves, rather than it’s type/class.

__If it talks and walks like a duck, then it is a duck__

So if we have an object that can fly and quack like a duck, we consider it as a duck. This called “Duck Typing”. In runtime, instead of checking the type of an object, we try to invoke a method we expect the object to have. If it behaves the way we expected, we’re fine and move along. But if it doesn’t, things might blow up. To be safe, we often handle the exceptions in a try..except block or use hasattr to check if an object has the specific method.


In the Python world, we often hear “file like object” or “an iterable” - if an object has a read method, it can be treated as a file like object, if it has an \_\_iter\_\_ magic method, it is an iterable. So any object, regardless of it’s class/type, can conform to a certain interface just by implementing the expected behavior (methods). These informal interfaces are termed as protocols. Since they are informal, they can not be formally enforced. They are mostly illustrated in the documentations or defined by convention. All the cool magic methods you have heard about - \_\_len\_\_, \_\_contains\_\_, \_\_iter\_\_ - they all help an object to conform to some sort of protocols.



In [0]:
class Team:
    def __init__(self, members):
        self.__members = members

    def __len__(self):
        return len(self.__members)

    def __contains__(self, member):
        return member in self.__members


justice_league_fav = Team(["batman", "wonder woman", "flash"])

# Sized protocol
print(len(justice_league_fav))

# Container protocol
print("batman" in justice_league_fav)
print("superman" in justice_league_fav)
print("cyborg" not in justice_league_fav)

3
True
False
True


In our above example, by implementing the \_\_len\_\_ and \_\_contains\_\_ method, we can now directly use the len function on a Team instance and check for membership using the in operator. If we add the \_\_iter\_\_ method to implement the iterable protocol, we would even be able to do something like:

In [0]:
for member in justice_league_fav:
    print(member)

Without implementing the \_\_iter\_\_ method, if we try to iterate over the team, we will get an error like:

TypeError: 'Team' object is not iterable
So we can see that protocols are like informal interfaces. We can implement a protocol by implementing the methods expected by it.

## Formal Interfaces: ABCs

While protocols work fine in many cases, there are situations where informal interfaces or duck typing in general can cause confusion. For example, a Bird and Aeroplane both can fly(). But they are not the same thing even if they implement the same interfaces / protocols. Abstract Base Classes or ABCs can help solve this issue.

The concept behind ABCs is simple - 
1. we define base classes which are abstract in nature. 
2. We define certain methods on the base classes as abstract methods. 

THEN:  __any objects deriving from these bases classes are forced to implement those methods.__ And since we’re using base classes, if we see an object has our class as a base class, we can say that this object implements the interface. That is now we can use types to tell if an object implements a certain interface. Let’s see an example.

In [0]:
import abc

class Bird(abc.ABC):
    @abc.abstractmethod
    def fly(self):
        pass

There’s the abc module which has a metaclass named ABCMeta. ABCs are created from this metaclass. So we can either use it directly as the metaclass of our ABC (something like this - class Bird(metaclass=abc.ABCMeta):) or we can subclass from the abc.ABC class which has the abc.ABCMeta as it’s metaclass already.

Then we have to use the abc.abstractmethod decorator to mark our methods abstract. 

Important: __if any class derives from our base Bird class, it must implement the fly method too.__ The following code would fail:

In [0]:
class Parrot(Bird):
    pass

p = Parrot()


TypeError: ignored

to fix it we write

In [0]:
class Parrot(Bird):
    def fly(self):
        print("Flying")


p = Parrot()

Now let’s define another ABC named Aeroplane like this:

In [0]:
class Aeroplane(abc.ABC):
    @abc.abstractmethod
    def fly(self):
        pass


class Boeing(Aeroplane):
    def fly(self):
        print("Flying!")

b = Boeing()

Now if we compare:

In [0]:
isinstance(p, Aeroplane)

False

In [0]:
isinstance(b, Bird)

False

We can see even though both objects have the same method fly but we can now differentiate easily which one implements the Bird interface and which implements the Aeroplane interface.

We saw how we can create our own, custom ABCs. But it is often discouraged to create custom ABCs and rather use/subclass the built in ones. The Python standard library has many useful ABCs that we can easily reuse. We can get a list of useful built in ABCs in the collections.abc module - https://docs.python.org/3/library/collections.abc.html#module-collections.abc. Before writing your own, please do check if there’s an ABC for the same purpose in the standard library.

## ABCs and Virtual Subclass
We can also register a class as a virtual subclass of an ABC. In that case, even if that class doesn’t subclass our ABC, it will still be treated as a subclass of the ABC (and thus accepted to have implemented the interface). Example codes will be able to demonstrate this better:

In [0]:
@Bird.register
class Robin:
    pass

r = Robin()

And then:



In [0]:
>>> issubclass(Robin, Bird)
True
>>> isinstance(r, Bird)
True
>>>

True

In this case, even if Robin does not subclass our ABC or define the abstract method, we can register it as a Bird. issubclass and isinstance behavior can be overloaded by adding two relevant magic methods. Read more on that here - https://www.python.org/dev/peps/pep-3119/#overloading-isinstance-and-issubclass

### Other Sources:

https://pymotw.com/3/abc/ 

https://bip.weizmann.ac.il/course/python/PyMOTW/PyMOTW/docs/abc/index.html 

# A new type of subclass¶
With the introduction of delegation-based instance and subclass checks, Python provides a new type of subclass, and thus a new way to relate classes together. Now a subclass may be a real subclass, obtained using inheritance

class ChildClass(ParentClass):
    pass
or can be a virtual subclass, obtained through registration

In [0]:
class ChildClass(ParentClass):
    pass

NameError: ignored

or can be a virtual subclass, obtained through registration


In [0]:
ParentClass.register(ChildClass)

NameError: ignored

### What is the  difference between a real and a virtual subclass?

a real subclass knows its relationship with the parent class through its \_\_bases\_\_ attribute, and can thus implicitly delegate the resolution of missing methods. A virtual subclass knows nothing about the class that registered it, and nowhere in the subclass will you find something that links it to the parent class. Thus, a virtual parent class is useful only as a categorization.

### Abstract Base Classes¶
Classes that can register other classes, thus becoming virtual parents of those, are called in Python Abstract Base Classes, or ABCs.

The name of this new language element is important. ABCs are first of all classes, just like any other class you can create in Python, and they can be subclassed in the usual way to create taxonomies. They are also meant to be base classes, that is classes that represent fundamental behaviours or categories. Last, they are abstract. This has a very precise meaning in Python and will be the subject of the last part of this post.

The classes provided by the collections module are Abstract Base Classes, and they set themselves as virtual parents of some base types in the same module. If you check the _collections_abc.py file in your Python 3 installation (for example in /usr/lib/python3.4/_collections_abc.py) you will find code like this

In [0]:
[...]

Sequence.register(tuple)
Sequence.register(str)
Sequence.register(range)

[...]

MutableSequence.register(list)

[...]

ModuleNotFoundError: ignored

Where the Sequence and the MutableSequence ABCs register some built-in types of Python.

__It is very important to understand that registering a class does not imply any form of check about methods or attributes. Registering is just the promise that a given behaviour is provided by the registered class.__

To demonstrate this let me provide you a very simple example made using one of the collections classes